In [1]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅


In [ ]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

In [ ]:
from subset import create_subset

subset, subset_name = create_subset(utility, utility_name)

In [2]:
from inference import generate_responses

responses, generation_name = generate_responses(prompts, "microsoft/Phi-3-mini-128k-instruct", ds_name, 'cuda:3', batch_size=64)

Generation: mix-instruct_train_21000_Phi-3-mini-128k-instruct found in cache, loading from cache ✅


In [5]:
from evaluation import compute_metrics

compute_metrics(responses, references, generation_name, device='cuda:0')

Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_rouge found in cache, loading from cache ✅


{'bge': 0.808487, 'rouge': 0.21496158928534306}

In [13]:
compute_metrics(responses, references, generation_name, device='cuda:0')

Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Phi-3-mini-128k-instruct_rouge found in cache, loading from cache ✅


{'bge': 0.808487, 'rouge': 0.21496158928534306}